In [0]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [0]:
import os
 
dirpath = os.getcwd()
print("current directory is : " + dirpath)

current directory is : /content


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# textgeneration.txt
# /content/gdrive/My Drive/DL/Project/textgeneration.txt

text=(open("/content/gdrive/My Drive/DL/Project/shakespear.txt").read())
text=text.lower()

In [0]:
characters = sorted(list(set(text)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [0]:
X = []
Y = []
length = len(text)
seq_length = 100

for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [0]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [0]:
X_modified, Y_modified

(array([[[0.5       ],
         [0.        ],
         [0.        ],
         ...,
         [0.27777778],
         [0.77777778],
         [0.02777778]],
 
        [[0.        ],
         [0.        ],
         [0.02777778],
         ...,
         [0.77777778],
         [0.02777778],
         [0.80555556]],
 
        [[0.        ],
         [0.02777778],
         [0.41666667],
         ...,
         [0.02777778],
         [0.80555556],
         [0.47222222]],
 
        ...,
 
        [[0.36111111],
         [0.02777778],
         [0.30555556],
         ...,
         [0.77777778],
         [0.02777778],
         [0.38888889]],
 
        [[0.02777778],
         [0.30555556],
         [0.38888889],
         ...,
         [0.02777778],
         [0.38888889],
         [0.63888889]],
 
        [[0.30555556],
         [0.38888889],
         [0.41666667],
         ...,
         [0.38888889],
         [0.63888889],
         [0.36111111]]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 

In [0]:
model = Sequential()
model.add(LSTM(600, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(600))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 600)          1444800   
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 600)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 600)               2882400   
_________________________________________________________________
dropout_4 (Dropout)          (None, 600)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 36)                21636     
Total params: 4,348,836
Trainable params: 4,348,836
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_modified, Y_modified, epochs=30, batch_size=50)

Epoch 1/30
19188/19188 [==============================] - 263s 14ms/step - loss: 3.0233
Epoch 2/30
19188/19188 [==============================] - 258s 13ms/step - loss: 2.9222
Epoch 3/30
19188/19188 [==============================] - 260s 14ms/step - loss: 2.7447
Epoch 4/30
19188/19188 [==============================] - 259s 13ms/step - loss: 2.6433
Epoch 5/30
19188/19188 [==============================] - 259s 13ms/step - loss: 2.5431
Epoch 6/30
19188/19188 [==============================] - 259s 13ms/step - loss: 2.4646
Epoch 7/30
19188/19188 [==============================] - 255s 13ms/step - loss: 2.3848
Epoch 8/30
19188/19188 [==============================] - 250s 13ms/step - loss: 2.3139
Epoch 9/30
19188/19188 [==============================] - 251s 13ms/step - loss: 2.2311
Epoch 10/30
19188/19188 [==============================] - 251s 13ms/step - loss: 2.1516
Epoch 11/30
19188/19188 [==============================] - 252s 13ms/step - loss: 2.0477
Epoch 12/30
19188/19188 [=====

In [0]:
#model 1
model.save_weights('/content/gdrive/My Drive/DL/Project/new_poem_weights/wider_model_30_epoch')

In [0]:
model.load_weights('/content/gdrive/My Drive/DL/Project/new_poem_weights/wider_model_30_epoch')

In [0]:
string_mapped = X[99]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]


In [0]:
#combining text
txt=""
for char in full_string:
    txt = txt+char
txt

" the riper should by time decease,\n his tender heir might bear his memory:\n but thou, contracted to thine own bright eyes,\n feed'st thy light's flame with self-substantial fuel,\n making a famine where abundance lies,\n thy self thy foe, to thy sweet self too cruel:\n thou that art now the world's fresh ornament,\n and only herald to the gaudy spring,\n within thine own bud buriest thy content,\n and tender churl mak'st waste in niggarding:\n   pity the world, or else this glutton be,\n   to eat the wor"

In [0]:
'''
" the riper should by time decease,\n
his tender heir might bear his memory:\n
but thou, contracted to thine own bright eyes,\n
feed'st thy light's flame with
self-substantial fuel,\n
making a famine where abundance lies,\n
thy self thy foe, to thy sweet self too cruel:\n
thou that art now the world's fresh ornament,\n
and only herald to the gaudy spring,\n
within thine own bud buriest thy content,\n
and tender churl mak'st waste in niggarding:\n
pity the world, or else this glutton be,\n
to eat the wor"

'''

'\n" the riper should by time decease,\n his tender heir might bear his memory:\n\nbut thou, contracted to thine own bright eyes,\n feed\'st thy light\'s flame with\nself-substantial fuel,\n making a famine where abundance lies,\n thy self thy\nfoe, to thy sweet self too cruel:\n thou that art now the world\'s fresh \nornament,\n and only herald to the gaudy spring,\n within thine own bud \nburiest thy content,\n and tender churl mak\'st waste in niggarding:\n  \npity the world, or else this glutton be,\n   to eat the wor"\n\n\n\n\n'

In [0]:
string_mapped3 = X[99]
full_string3 = [n_to_char[value] for value in string_mapped3]
# generating characters
for i in range(800):
    x = np.reshape(string_mapped3,(1,len(string_mapped3), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped3]
    full_string3.append(n_to_char[pred_index])

    string_mapped3.append(pred_index)
    string_mapped3 = string_mapped3[1:len(string_mapped3)]

In [0]:
txt3=""
for char in full_string3:
    txt3 = txt3+char
txt3

" the riper should by time decease,\n his tender heir might bear his memory:\n but thou, contracted to thine own bright eyes,\n feed'st thy light's flame with self-substantial fuel,\n making a famine where abundance lies,\n thy self thy foe, to thy sweet self too cruel:\n thou that art now the world's fresh ornament,\n and only herald to the gaudy spring,\n within thine own bud buriest thy content,\n and tender churl mak'st waste in niggarding:\n   pity the world, or else this glutton be,\n   to eat the world's due, by the grave and thee.\n\n ii\n\n when forty winters shall besiege thy brow,\n and dig deep trenches in thy beauty's field,\n thy youth's proud livery so gazed on now,\n will be a tatter'd weed of small worth held:\n then being asked, where all thy beauty lies,\n where all the treasure of thy lusty days;\n to say, within thine own deep sunken eyes,\n were an all-eating shame, and thriftless praise."

In [0]:
'''
" the riper should by time decease,\n
his tender heir might bear his memory:\n
but thou, contracted to thine own bright eyes,\n
feed'st thy light's flame with self-substantial fuel,\n
making a famine where abundance lies,\n
thy self thy foe, to thy sweet self too cruel:\n
thou that art now the world's fresh ornament,\n
and only herald to the gaudy spring,\n
within thine own bud buriest thy content,\n
and tender churl mak'st waste in niggarding:\n
pity the world, or else this glutton be,\n 
to eat the world's due, by the grave and thee.
\n\n ii\n\n
when forty winters shall besiege thy brow,\n
and dig deep trenches in thy beauty's field,\n
thy youth's proud livery so gazed on now,\n
will be a tatter'd weed of small worth held:\n
then being asked, where all thy beauty lies,\n
where all the treasure of thy lusty days;\n
to say, within thine own deep sunken eyes,\n 
were an all-eating shame, and thriftless praise."



'''